In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Pernambuco - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Pernambuco - Produção de Cimento (t)', axis=1)
data

,Ano,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Pernambuco - Consumo de Cimento (t)
0,2008,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,0.704795,0.787611,0.660899,5.483335e+08,2.546644e+09,...,8.049769,0.671922,23.704331,1012.356520,1564.281549,25.868467,21.249870,75.428608,2.118063,1717.153000
1,2009,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,0.707057,0.790400,0.665481,5.302764e+08,2.679945e+09,...,8.078293,0.671804,22.861798,1012.209502,1487.098457,25.873200,21.577129,77.771392,1.951534,1878.938000
2,2010,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,0.709020,0.792674,0.669877,5.046425e+08,2.891930e+09,...,8.042187,0.671411,22.487970,1012.234225,1437.305833,25.750070,21.891970,80.133165,1.854189,2406.931000
3,2011,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,0.710693,0.794444,0.674084,4.854622e+08,3.091988e+09,...,8.028882,0.670748,21.107295,1012.512701,1436.247207,25.736127,21.841067,80.179132,1.815199,2496.113000
4,2012,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,0.711600,0.795572,0.677443,4.867661e+08,3.189506e+09,...,12.192814,0.669292,21.566683,1012.689602,1436.457629,25.711228,21.251350,77.643662,1.728413,2728.230000
5,2013,9.627386e+07,5.616792e+06,8.939248,7.935398e+07,0.713567,0.796536,0.682563,5.225848e+08,3.093872e+09,...,13.117303,0.668984,21.669594,1012.907542,1521.117838,25.685608,20.953182,76.283825,1.611013,2982.021000
6,2014,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.714447,0.796914,0.686026,5.676413e+08,2.779340e+09,...,13.857509,0.667649,19.852601,1013.159804,1654.440407,25.780576,20.818540,75.113876,1.558118,2983.632857
7,2015,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.714401,0.796740,0.688529,5.179392e+08,2.561731e+09,...,14.543115,0.665120,18.352626,1013.187593,1731.551083,25.909409,21.066299,75.720358,1.597579,2632.837886
8,2016,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.722910,0.809069,0.701506,4.524144e+08,2.516859e+09,...,14.833751,0.666277,16.821327,1013.081354,1774.985187,25.897163,21.315125,76.984087,1.606812,2308.455257
9,2017,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.722811,0.808712,0.702551,4.213689e+08,2.497710e+09,...,14.580259,0.665302,16.755645,1013.025277,1768.623220,25.854367,21.296659,77.129112,1.578862,1813.941000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,8.785657e+07,5.769035e+06,8.834432,7.768216e+07,0.704795,0.787611,0.660899,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,8.049769,0.671922,23.704331,1012.356520,1564.281549,25.868467,21.249870,75.428608,2.118063
1,8.860216e+07,5.810149e+06,8.858245,7.790384e+07,0.707057,0.790400,0.665481,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,8.078293,0.671804,22.861798,1012.209502,1487.098457,25.873200,21.577129,77.771392,1.951534
2,9.096526e+07,5.826469e+06,8.909890,7.856512e+07,0.709020,0.792674,0.669877,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,8.042187,0.671411,22.487970,1012.234225,1437.305833,25.750070,21.891970,80.133165,1.854189
3,9.301827e+07,5.792701e+06,8.937706,7.897343e+07,0.710693,0.794444,0.674084,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,8.028882,0.670748,21.107295,1012.512701,1436.247207,25.736127,21.841067,80.179132,1.815199
4,9.484266e+07,5.717659e+06,8.949260,7.926901e+07,0.711600,0.795572,0.677443,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,12.192814,0.669292,21.566683,1012.689602,1436.457629,25.711228,21.251350,77.643662,1.728413
5,9.627386e+07,5.616792e+06,8.939248,7.935398e+07,0.713567,0.796536,0.682563,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,13.117303,0.668984,21.669594,1012.907542,1521.117838,25.685608,20.953182,76.283825,1.611013
6,9.733647e+07,5.498393e+06,8.923227,7.929548e+07,0.714447,0.796914,0.686026,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,13.857509,0.667649,19.852601,1013.159804,1654.440407,25.780576,20.818540,75.113876,1.558118
7,9.797155e+07,5.411870e+06,8.896326,7.910750e+07,0.714401,0.796740,0.688529,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,14.543115,0.665120,18.352626,1013.187593,1731.551083,25.909409,21.066299,75.720358,1.597579
8,9.849009e+07,5.357988e+06,8.887508,7.905537e+07,0.722910,0.809069,0.701506,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,14.833751,0.666277,16.821327,1013.081354,1774.985187,25.897163,21.315125,76.984087,1.606812
9,9.859294e+07,5.346247e+06,8.880979,7.898921e+07,0.722811,0.808712,0.702551,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,14.580259,0.665302,16.755645,1013.025277,1768.623220,25.854367,21.296659,77.129112,1.578862


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1878.938000
1     2406.931000
2     2496.113000
3     2728.230000
4     2982.021000
5     2983.632857
6     2632.837886
7     2308.455257
8     1813.941000
9     1645.004000
10    1690.846000
11    2018.834000
12    2022.411000
13    1940.927000
14            NaN
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.894045,1.016308,-1.508819,-2.131804,-1.583249,-1.530584,-1.598164,1.204752,-0.990234,0.166091,...,0.984445,-1.373812,1.355095,1.532738,-1.179403,-0.231175,0.290203,-0.299372,-1.157338,1.628734
1,-1.694920,1.229887,-0.890758,-1.702987,-1.220442,-1.140159,-1.290136,0.820657,-0.546688,0.428911,...,-0.051710,-1.363879,1.312892,1.177660,-1.613673,-0.847777,0.336507,0.759984,0.359687,0.803272
2,-1.063798,1.314665,0.449724,-0.423797,-0.905536,-0.821949,-0.994665,0.275392,0.158672,0.847995,...,-0.115145,-1.376452,1.172184,1.020114,-1.540643,-1.245561,-0.868108,1.779145,1.889008,0.320744
3,-0.515495,1.139248,1.171687,0.366041,-0.637043,-0.574308,-0.711867,-0.132596,0.824346,1.230291,...,0.633403,-1.381086,0.934380,0.438242,-0.718070,-1.254018,-1.004513,1.614369,1.918773,0.127477
4,-0.028247,0.749420,1.471589,0.937804,-0.491590,-0.416427,-0.486099,-0.104860,1.148829,1.382743,...,-0.646476,0.068995,0.412716,0.631847,-0.195530,-1.252337,-1.248108,-0.294581,0.276978,-0.302714
5,0.353986,0.225440,1.211705,1.102177,-0.176084,-0.281472,-0.141927,0.657046,0.830615,1.112297,...,-0.757614,0.390946,0.301975,0.675218,0.448231,-0.576002,-1.498756,-1.259768,-0.603559,-0.884651
6,0.637781,-0.389615,0.795877,0.989019,-0.034819,-0.228569,0.090888,1.615453,-0.215960,0.282052,...,0.345772,0.648721,-0.176426,-0.090537,1.193372,0.489088,-0.569662,-1.695614,-1.361137,-1.146844
7,0.807393,-0.839081,0.097662,0.625383,-0.042209,-0.252893,0.259082,0.558230,-0.940033,-0.632425,...,1.298413,0.887482,-1.082994,-0.722687,1.275456,1.105111,0.690753,-0.893603,-0.968421,-0.951240
8,0.945883,-1.118984,-0.131209,0.524538,1.322915,1.472501,1.131361,-0.835563,-1.089341,-1.093812,...,1.596124,0.988695,-0.668215,-1.368038,0.961643,1.452098,0.570944,-0.088137,-0.150117,-0.905473
9,0.973351,-1.179977,-0.300688,0.396558,1.307025,1.422494,1.201669,-1.495939,-1.153056,-1.335633,...,-0.048438,0.900417,-1.017629,-1.395719,0.796001,1.401274,0.152257,-0.147914,-0.056209,-1.044017


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.63530724, -0.95624748, -1.14206076, -0.46989042,
          1.08697714,  1.02203982,  1.12974224, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          0.74875685, -1

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     1878.938000
1     2406.931000
2     2496.113000
3     2728.230000
4     2982.021000
5     2983.632857
6     2632.837886
7     2308.455257
8     1813.941000
9     1645.004000
10    1690.846000
11    2018.834000
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    2018.834
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.243152,-0.752383,-1.854042,-1.166663,0.818285,0.699365,1.038092,1.108405,2.635823,-0.189959,...,-2.260447,0.742044,-1.580713,-1.117166,-0.348766,-0.031355,0.055743,1.110593,1.231105,0.854739


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.24315161, -0.7523835 , -1.85404211, -1.16666318,
          0.81828459,  0.69936487,  1.03809159,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          0.74204352, -1

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    2022.411
Name: Pernambuco - Consumo de Cimento (t), dtype: float64

In [30]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [31]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [32]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3518656415, 1907982345, 1135124863, 106710060, 629650363, 3962283708, 3764449037, 4216811133, 909983667, 3115777540, 2804111292, 949955841, 1820039250, 4192032552, 406938380, 3888499163, 2427984169, 1389509328, 1421198699, 2475104463, 1208260337, 3669143884, 4119150764, 1424141859, 4068356193, 1035115600, 2195796936, 2378686690, 1564562754, 2348453989, 811172287, 4171791578, 4167034435, 3716834300, 2619612706, 2702519155, 403731441, 1375519715, 74305237, 222614267, 2543095268, 1059581343, 822656034, 830589544, 1598000665, 2057357227, 857503465, 4225346347, 4014646186, 3242484039, 35977891, 4153115586, 1575102227, 3136212086, 2828144709, 3181343464, 77283685, 2357790519, 292221345, 1239490567, 1499303990, 78660703, 1091419836, 1359406018, 2459169929, 3905493123, 3362158441, 205391280, 1065531338, 4005555017, 1947658772, 1340859733, 1524380508, 1286634188, 4265256219, 2704160349, 2906693012, 3291921138, 1115729625, 1812846565, 1068047114, 34611990, 2870041286, 3159406695, 530096037, 234

loss: 17827.587890625


Step: 91 ___________________________________________
loss: 30462.515625


Step: 92 ___________________________________________
loss: 26198.328125


Step: 93 ___________________________________________
loss: 21910.21875


Step: 94 ___________________________________________
loss: 16867.453125


Step: 95 ___________________________________________
loss: 22039.572265625


Step: 96 ___________________________________________
loss: 24686.9140625


Step: 97 ___________________________________________
loss: 37995.9609375


Step: 98 ___________________________________________
loss: 12220.8779296875


Step: 99 ___________________________________________
loss: 12120.9853515625


final_seed: 1564562754


In [33]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 5498669.0000
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 5265478.5000
Epoch 3/10000
1/1 [==============================] - 0s 7ms/step - loss: 1793275.6250
Epoch 4/10000
1/1 [==============================] - 0s 7ms/step - loss: 1298058.3750
Epoch 5/10000
1/1 [==============================] - 0s 4ms/step - loss: 2117001.5000
Epoch 6/10000
1/1 [==============================] - 0s 7ms/step - loss: 297068.7500
Epoch 7/10000
1/1 [==============================] - 0s 6ms/step - loss: 315633.6562
Epoch 8/10000
1/1 [==============================] - 0s 7ms/step - loss: 156326.3906
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 272074.8438
Epoch 10/10000
1/1 [==============================] - 0s 8ms/step - loss: 88018.4375
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 126423.5391
Epoch 12/10000
1/1 [==============================] - 0s 7ms

1/1 [==============================] - 0s 6ms/step - loss: 73783.6641
Epoch 97/10000
1/1 [==============================] - 0s 5ms/step - loss: 82778.4609
Epoch 98/10000
1/1 [==============================] - 0s 8ms/step - loss: 34883.8711
Epoch 99/10000
1/1 [==============================] - 0s 5ms/step - loss: 80103.7109
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 41921.6094
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 115689.7734
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 112465.5859
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 100942.6797
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 37696.8477
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 96823.8984
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 55398.0820
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 49967.1562
Epoch 192/10000
1/1 [==============================] - 0s 4ms/step - loss: 67156.0625
Epoch 193/10000
1/1 [==============================] - 0s 4ms/step - loss: 83120.5469
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 39737.2656
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 40013.1562
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 81228.2500
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 58200.5938
Epoch 198/10000
1/1 [==============================] - 0s 5ms/step - loss: 99088.1172
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 95070.6484
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 92170.8516
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 124167.4609
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - 

1/1 [==============================] - 0s 4ms/step - loss: 107255.6172
Epoch 287/10000
1/1 [==============================] - 0s 4ms/step - loss: 49420.0195
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 81634.8281
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 61795.2812
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 95940.1016
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 90018.3984
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 100764.2734
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 45031.9062
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 54570.9648
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 68595.7578
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 63558.0430
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step -

1/1 [==============================] - 0s 4ms/step - loss: 45320.5117
Epoch 382/10000
1/1 [==============================] - 0s 4ms/step - loss: 43084.8867
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 32537.8809
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 47949.2188
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 54320.3555
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 29370.7051
Epoch 387/10000
1/1 [==============================] - 0s 7ms/step - loss: 19533.7598
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 76753.5547
Epoch 389/10000
1/1 [==============================] - 0s 5ms/step - loss: 83190.1641
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 80883.3594
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 55486.6367
Epoch 392/10000
1/1 [==============================] - 0s 5ms/step - l

...

1/1 [==============================] - 0s 4ms/step - loss: 27536.9434
Epoch 857/10000
1/1 [==============================] - 0s 4ms/step - loss: 63137.2812
Epoch 858/10000
1/1 [==============================] - 0s 5ms/step - loss: 95116.7734
Epoch 859/10000
1/1 [==============================] - 0s 5ms/step - loss: 53178.7305
Epoch 860/10000
1/1 [==============================] - 0s 4ms/step - loss: 70850.7109
Epoch 861/10000
1/1 [==============================] - 0s 5ms/step - loss: 98212.0625
Epoch 862/10000
1/1 [==============================] - 0s 4ms/step - loss: 70904.8984
Epoch 863/10000
1/1 [==============================] - 0s 4ms/step - loss: 63800.8438
Epoch 864/10000
1/1 [==============================] - 0s 4ms/step - loss: 79242.0312
Epoch 865/10000
1/1 [==============================] - 0s 4ms/step - loss: 92012.7891
Epoch 866/10000
1/1 [==============================] - 0s 4ms/step - loss: 87126.6641
Epoch 867/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 78025.7500
Epoch 952/10000
1/1 [==============================] - 0s 4ms/step - loss: 62781.7188
Epoch 953/10000
1/1 [==============================] - 0s 4ms/step - loss: 33330.7148
Epoch 954/10000
1/1 [==============================] - 0s 4ms/step - loss: 102085.8359
Epoch 955/10000
1/1 [==============================] - 0s 6ms/step - loss: 89592.5938
Epoch 956/10000
1/1 [==============================] - 0s 4ms/step - loss: 88191.8438
Epoch 957/10000
1/1 [==============================] - 0s 4ms/step - loss: 75540.2109
Epoch 958/10000
1/1 [==============================] - 0s 4ms/step - loss: 81675.4453
Epoch 959/10000
1/1 [==============================] - 0s 4ms/step - loss: 51384.1445
Epoch 960/10000
1/1 [==============================] - 0s 4ms/step - loss: 70098.7266
Epoch 961/10000
1/1 [==============================] - 0s 4ms/step - loss: 49866.6367
Epoch 962/10000
1/1 [==============================] - 0s 5ms/step - 

1/1 [==============================] - 0s 4ms/step - loss: 38420.0117
Epoch 1046/10000
1/1 [==============================] - 0s 4ms/step - loss: 38198.7695
Epoch 1047/10000
1/1 [==============================] - 0s 4ms/step - loss: 66882.5938
Epoch 1048/10000
1/1 [==============================] - 0s 4ms/step - loss: 66005.6016
Epoch 1049/10000
1/1 [==============================] - 0s 4ms/step - loss: 47934.0625
Epoch 1050/10000
1/1 [==============================] - 0s 5ms/step - loss: 78990.1250
Epoch 1051/10000
1/1 [==============================] - 0s 4ms/step - loss: 62263.3750
Epoch 1052/10000
1/1 [==============================] - 0s 4ms/step - loss: 34127.6055
Epoch 1053/10000
1/1 [==============================] - 0s 5ms/step - loss: 43426.0195
Epoch 1054/10000
1/1 [==============================] - 0s 4ms/step - loss: 56627.9727
Epoch 1055/10000
1/1 [==============================] - 0s 4ms/step - loss: 69543.0703
Epoch 1056/10000
1/1 [==============================] - 0s 5

1/1 [==============================] - 0s 4ms/step - loss: 41745.8125
Epoch 1140/10000
1/1 [==============================] - 0s 4ms/step - loss: 40961.3945
Epoch 1141/10000
1/1 [==============================] - 0s 5ms/step - loss: 90585.3828
Epoch 1142/10000
1/1 [==============================] - 0s 4ms/step - loss: 53207.3789
Epoch 1143/10000
1/1 [==============================] - 0s 5ms/step - loss: 45741.8086
Epoch 1144/10000
1/1 [==============================] - 0s 6ms/step - loss: 47413.6250
Epoch 1145/10000
1/1 [==============================] - 0s 4ms/step - loss: 90606.7578
Epoch 1146/10000
1/1 [==============================] - 0s 5ms/step - loss: 57234.1758
Epoch 1147/10000
1/1 [==============================] - 0s 5ms/step - loss: 67953.3359
Epoch 1148/10000
1/1 [==============================] - 0s 5ms/step - loss: 84808.2812
Epoch 1149/10000
1/1 [==============================] - 0s 4ms/step - loss: 65105.4375
Epoch 1150/10000
1/1 [==============================] - 0s 5

1/1 [==============================] - 0s 4ms/step - loss: 43215.5586
Epoch 1234/10000
1/1 [==============================] - 0s 4ms/step - loss: 53554.6758
Epoch 1235/10000
1/1 [==============================] - 0s 4ms/step - loss: 66938.9609
Epoch 1236/10000
1/1 [==============================] - 0s 5ms/step - loss: 49537.4570
Epoch 1237/10000
1/1 [==============================] - 0s 4ms/step - loss: 38747.9844
Epoch 1238/10000
1/1 [==============================] - 0s 4ms/step - loss: 43679.3867
Epoch 1239/10000
1/1 [==============================] - 0s 5ms/step - loss: 69320.9141
Epoch 1240/10000
1/1 [==============================] - 0s 5ms/step - loss: 49396.1875
Epoch 1241/10000
1/1 [==============================] - 0s 4ms/step - loss: 85769.7109
Epoch 1242/10000
1/1 [==============================] - 0s 4ms/step - loss: 58987.9844
Epoch 1243/10000
1/1 [==============================] - 0s 5ms/step - loss: 42186.7031
Epoch 1244/10000
1/1 [==============================] - 0s 4

In [34]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 595ms/step
[2022.411](test_target) - [[1934.0388]](prediction) = 88.37218164062506


In [35]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.043696450247068995

In [36]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [37]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-0.969522,-1.359398,-1.050601,-0.981735,-1.252557,-1.264163,-1.233150,1.148205,-1.122043,-1.121418,...,1.412346,-0.449079,0.959570,1.349075,1.396668,1.302745,0.665154,-1.232563,-1.223094,1.316781
1,-0.406879,0.342020,-0.294564,-0.390692,0.057661,0.083064,0.016987,0.140893,-0.184474,-0.185492,...,-0.643250,1.385895,0.419892,-0.307116,-0.890660,-0.174780,0.748246,0.015789,-0.003295,-0.211662
2,1.376400,1.017379,1.345165,1.372427,1.194896,1.181098,1.216163,-1.289098,1.306518,1.306910,...,-0.769096,-0.936816,-1.379462,-1.041959,-0.506008,-1.127966,-1.413400,1.216774,1.226389,-1.105119


train_target:


,Pernambuco - Consumo de Cimento (t)
0,1878.938
1,2406.931
2,2496.113


test_input:


...

,Pernambuco - PIB - Estadual,Pernambuco - PIB - Construção Civil,Pernambuco - PIB - Per Capita,Pernambuco - PIB - Preços de Mercado,Pernambuco - IDH,Pernambuco - IDH Longevidade,Pernambuco - IDH Educacao,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Pernambuco - Desemprego,Pernambuco - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,0.243152,-0.752383,-1.854042,-1.166663,0.818285,0.699365,1.038092,1.108405,2.635823,-0.189959,...,-2.260447,0.742044,-1.580713,-1.117166,-0.348766,-0.031355,0.055743,1.110593,1.231105,0.854739


test_target:


,Pernambuco - Consumo de Cimento (t)
12,2022.411


1/1 [==============================] - 1s 588ms/step
Error: 88.37218164062506




[2456.13330078125,
 2589.572265625,
 2988.653564453125,
 2853.64599609375,
 2562.230712890625,
 2283.48046875,
 1982.5948486328125,
 1960.25244140625,
 1794.7598876953125,
 1934.038818359375]

In [38]:
display(targets)
display(predictions)

[2728.23,
 2982.021,
 2983.632857142857,
 2632.8378857142857,
 2308.455257142857,
 1813.941,
 1645.004,
 1690.846,
 2018.834,
 2022.411]

[2456.13330078125,
 2589.572265625,
 2988.653564453125,
 2853.64599609375,
 2562.230712890625,
 2283.48046875,
 1982.5948486328125,
 1960.25244140625,
 1794.7598876953125,
 1934.038818359375]

In [39]:
mae = mean_absolute_error(predictions, targets)
mae

253.31327597656255

In [40]:
porcentage = mae/np.mean(targets)
porcentage

0.11097472715976259